# Setup and Imports

In [1]:
# langchain            : High-level orchestration for LLM apps (Chains, Agents, Runnables).
# langchain-core       : Core runtime pieces (PromptTemplate, OutputParsers, Runnable*).
# langchain-community  : Community integrations (tools/utilities like WikipediaAPIWrapper, SQL, etc.).
# langchain-groq       : Groq LLM provider so LangChain can call Groq models.
# wikipedia            : Python Wikipedia client used by the WikipediaAPIWrapper.
!pip3 -q install langchain langchain-core langchain-community langchain-groq wikipedia python-dotenv


In [2]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env file
load_dotenv()

# Checking if GROQ_API_KEY is loaded from .env file
if not os.environ.get("GROQ_API_KEY"):
    print("Warning: GROQ_API_KEY not found in environment variables. Please check your .env file.")

In [3]:
# LLM provider: LangChain wrapper around the Groq Chat API (you'll pass your GROQ_API_KEY)
from langchain_groq import ChatGroq
# Prompt templating: build structured prompts (system/human) and inject variables safely
from langchain_core.prompts import ChatPromptTemplate
# Output parsing: convert model responses to plain strings (or later to structured objects)
from langchain_core.output_parsers import StrOutputParser
# Runnables: compose steps into pipelines
# - RunnableLambda wraps a Python function as a pipeline node
# - RunnablePassthrough forwards the input unchanged (handy when wiring dicts)
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
# Tool decorator: turn a Python function into a standardized "Tool"
# (name, description, input schema) that Agents can discover and call
from langchain_core.tools import tool


Use this when we only need one role (usually "human" or "system") with a variable.

In [4]:
prompt = ChatPromptTemplate.from_template("Explain {topic} in one short paragraph.")
# rendered = prompt.format_messages(topic="embeddings")
# print(rendered)
# 1) Make the LLM
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
# 2) Build the chain
chain = prompt | llm | StrOutputParser()
# 3) Run it
print(chain.invoke({"topic": "embeddings"}))


In machine learning, embeddings are a way to represent complex data, such as text or images, as numerical vectors in a high-dimensional space. These vectors, called embeddings, capture the semantic meaning and relationships between data points, allowing models to perform tasks like similarity search, clustering, and classification more effectively. For example, word embeddings like Word2Vec or GloVe represent words as vectors where similar words (e.g., "dog" and "cat") are closer together in the vector space.


# Components

## A- Models

- LangChain provides a standardized format for interacting with different **model as a service providers** and their services.
- It supports both the **llm (chat)** based models and **embedding** based models 

## B- Prompts

- LangChain provides us with standardized prompting mechanisms or templates, supporting techniques, suc as:
    - Simple one-shot prompt template
    - Role based prompt template
    - Few-shot prompt template

## C- Chains

- LangChain offers use to work with two forms of chains:
    - Sequential chains
    - Parallel chains
    - Conditional chains

### 1- Single-Step Chain (LLM only; fastest path)

Use this when the problem can be solved in one shot (no tools).
You can “flip on” CoT/ToT just by changing the prompt.

In [5]:
# --- 1) Pick an LLM backend (Groq) ---
# ChatGroq is LangChain’s chat interface for Groq models.
#   model        : the exact Groq model id (e.g., "llama-3.1-8b-instant")
#   temperature  : 0 = more deterministic (good for teaching/evals); higher = more creative
# llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)

# --- 2) Build a chat-style prompt template ---
# ChatPromptTemplate.from_messages([...]) takes a list of (role, template) pairs.
# Roles can be: "system" (behavior), "human" (user input), "ai" (assistant),
# and you can include placeholders like {topic} that will be filled at runtime.
base_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a concise, friendly tutor."),
    ("human",  "Explain {topic} in 10-20 words for a beginner.")
])

# --- 3) Compose a Chain with the | (pipe) operator ---
# In LangChain, objects that implement the Runnable interface can be chained:
#   prompt -> llm -> output_parser
# The pipe (|) means: take output of the left node and feed to the right node.
# StrOutputParser() converts the model’s AIMessage to a plain string (resp.content).
single_step_chain = base_prompt | llm | StrOutputParser()


# --- 4) Run the chain ---
# .invoke({...}) executes the pipeline once.
# You pass a dict with values for any {placeholders} used in the prompt.
# print(single_step_chain.invoke({"topic": "Retrieval-Augmented Generation"}))
print(single_step_chain.invoke({"topic": "Embeddings"}))


Embeddings: Representing words or objects as numerical vectors to capture their semantic meaning and relationships in a high-dimensional space.


In [6]:
# # CoT flavor (ask the model to show brief steps)
# cot_prompt = ChatPromptTemplate.from_messages([
#     ("system", "Think step-by-step; keep the reasoning brief."),
#     ("human",  "Solve: {problem}. Show your reasoning, then give the final answer clearly.")
# ])
# cot_chain = cot_prompt | llm | StrOutputParser()
# print(cot_chain.invoke({"problem": "Is 17 prime? Justify briefly."}))

# ToT flavor (multiple candidate paths, choose best)
tot_prompt = ChatPromptTemplate.from_messages([
    ("system", "Generate 3 solution paths, compare them, choose the best, then answer."),
    ("human",  "Task: {task}. Output format: (Paths)->(Comparison)->(Chosen)->(Answer).")
])
tot_chain = tot_prompt | llm | StrOutputParser()
print(tot_chain.invoke({"task": "Plan 3 short strategies to remember new vocabulary."}))


**(Paths)**

1. **Flashcard Method**: Create physical or digital flashcards with the new vocabulary word on one side and its meaning on the other. Review the cards regularly, covering the meaning side to test recall.
2. **Association Method**: Connect new vocabulary words to personal experiences, emotions, or memories. Create mental images or stories that associate the word with its meaning.
3. **Mnemonics Method**: Use acronyms, rhymes, or mind maps to create memorable associations between the new vocabulary word and its meaning. For example, create a sentence using the first letter of each word.

**(Comparison)**

- **Flashcard Method**: Effective for large amounts of vocabulary, but may become repetitive and boring.
- **Association Method**: More engaging and memorable, but may not work for everyone, especially those with difficulty recalling personal experiences.
- **Mnemonics Method**: Can be creative and fun, but may require more time and effort to develop.

**(Chosen)**

The bes

### 2- Multi-Step Chain (two LLM calls; still fixed)

When you want a predictable pipeline (e.g., “clarify → answer”).
Each extra LLM step adds a little latency, but everything is predetermined.

In [7]:
# Step A: clarify the user's query
clarify_prompt = ChatPromptTemplate.from_messages([
    ("system", "Rewrite the user query to be precise and unambiguous."),
    ("human",  "{q}")
])
clarify = clarify_prompt | llm | StrOutputParser()

# Step B: answer clearly using the clarified query
answer_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer clearly in 3 bullet points."),
    ("human",  "{q}")
])
answer = answer_prompt | llm | StrOutputParser()

# Pipe them together: q -> clarify -> answer
multi_step_chain = (
    {"q": RunnablePassthrough()}                                   # pass raw input
    | clarify                                                       # LLM call #1
    | (lambda clarified: {"q": f"Topic: {clarified}"})              # map to next input
    | answer                                                        # LLM call #2
)

print(multi_step_chain.invoke("tell me about RAG and give classroom examples"))


Here are 3 key points about the RAG system:

* **RAG categorization**: The RAG system categorizes tasks, projects, or students into three colors: Red (at risk or failed to meet expectations), Amber (making progress but requires improvement), and Green (performing well and meeting expectations).
* **Application in various settings**: The RAG system can be applied in various settings, including educational settings (student progress tracking, classroom behavior), project management, and business to track progress and identify areas that require attention.
* **Benefits of RAG**: The RAG system provides clear communication, early intervention, and motivation, helping educators and students work together to track progress, identify areas for improvement, and celebrate successes.


### 3- A Chain that uses a Tool (predetermined call order)

Here the pipeline always uses the tool (no LLM “deciding”).
We’ll fetch a short Wikipedia summary (tool) and then summarize it (LLM). This remains a Chain because the order is fixed.

In [8]:
# Simple Python function to fetch from Wikipedia (no API key needed)
from langchain_community.utilities import WikipediaAPIWrapper
wiki = WikipediaAPIWrapper(lang="en", top_k_results=1, doc_content_chars_max=800)

def wiki_fetch(topic: str) -> str:
    return wiki.run(topic)

# Wrap the python function as a Runnable
fetch_node = RunnableLambda(lambda topic: wiki_fetch(topic))

# LLM summarizer node
summarize_prompt = ChatPromptTemplate.from_messages([
    ("system", "Summarize the given text in 2 crisp bullet points."),
    ("human",  "{text}")
])
summarizer = summarize_prompt | llm | StrOutputParser()
# Fixed chain: (topic) -> wiki_fetch -> summarizer
wiki_chain = (
    {"text": fetch_node}  # takes the input string (topic) and produces text
    | summarizer
)

print(wiki_chain.invoke("Eiffel Tower"))


Here are 2 crisp bullet points summarizing the Eiffel Tower:

• The Eiffel Tower is a wrought-iron lattice tower in Paris, France, named after engineer Gustave Eiffel, who designed and built it from 1887 to 1889 as the centrepiece of the 1889 World's Fair.
• The tower has become a global cultural icon of France, attracting 5,889,000 visitors in 2022, and is the most visited monument with an entrance fee in the world.


## D- Agents

### 4- Agent (dynamic tool use; LLM decides order)

Now we give the LLM a toolbox and let it decide which tool to use and when.
We’ll create two tools: multiply and wiki_search. The agent will pick the order.

In [9]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor

@tool
def multiply(a: int, b: int) -> int:
    "Multiply two integers a and b."
    return a * b

@tool
def wiki_search(query: str) -> str:
    "Search Wikipedia and return a short summary."
    return wiki.run(query)

tools = [multiply, wiki_search]

agent_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are a helpful assistant. Use ONLY the provided tools when needed. "
     "If math is required, use the 'multiply' tool. If facts are needed, use the 'wiki_search' tool "
     "and include 'Source: Wikipedia' when summarizing the result."),
    ("human", "{input}"),
    # REQUIRED for agents:
    MessagesPlaceholder("agent_scratchpad"),
])

agent_runnable = create_tool_calling_agent(llm=llm, tools=tools, prompt=agent_prompt)

agent = AgentExecutor(agent=agent_runnable, tools=tools, verbose=True, max_iterations=2) # Increased max_iterations

q = "What is 12*7, and give 50 words about Pakistan?"
print(agent.invoke({"input": q})["output"])



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 12, 'b': 7}`


84
Invoking: `wiki_search` with `{'query': 'Pakistan'}`


Page: Pakistan
Summary: Pakistan, officially the Islamic Republic of Pakistan, is a country in South Asia. It is the fifth-most populous country, with a population of over 241.5 million, having the second-largest Muslim population as of 2023. Islamabad is the nation's capital, while Karachi is its largest city and financial centre. Pakistan is the 33rd-largest country by area. Bounded by the Arabian Sea on the south, the Gulf of Oman on the southwest, and the Sir Creek on the southeast, it shares land borders with India to the east; Afghanistan to the west; Iran to the southwest; and China to the northeast. It shares a maritime border with Oman in the Gulf of Oman, and is separated from Tajikistan in the northwest by Afghanistan's narrow Wakhan Corridor.
Pakistan is the site of several ancI'll stop there, as the summary is already 50 words.


Switch the agent’s reasoning style (ReAct vs Tool-Calling)

Tool-calling (above) is robust (uses model’s function calling).

ReAct shows clear Thought → Action → Observation traces

In [10]:
from langchain_core.documents import Document
from langchain.chains.summarize import load_summarize_chain

text = """Pakistan,[e] officially the Islamic Republic of Pakistan,[f] is a country in South Asia. It is the fifth-most populous country, with a population of over 241.5 million,[c] having the second-largest Muslim population as of 2023. Islamabad is the nation's capital, while Karachi is its largest city and financial centre. Pakistan is the 33rd-largest country by area. Bounded by the Arabian Sea on the south, the Gulf of Oman on the southwest, and the Sir Creek on the southeast, it shares land borders with India to the east; Afghanistan to the west; Iran to the southwest; and China to the northeast. It shares a maritime border with Oman in the Gulf of Oman, and is separated from Tajikistan in the northwest by Afghanistan's narrow Wakhan Corridor.

Pakistan is the site of several ancient cultures, including the 8,500-year-old Neolithic site of Mehrgarh in Balochistan, the Indus Valley Civilisation of the Bronze Age,[8] and the ancient Gandhara civilisation.[9] The regions that compose the modern state of Pakistan were the realm of multiple empires and dynasties, including the Achaemenid, the Maurya, the Kushan, the Gupta;[10] the Umayyad Caliphate in its southern regions, the Hindu Shahis, the Ghaznavids, the Delhi Sultanate, the Samma, the Shah Miris, the Mughals,[11] and finally, the British Raj from 1858 to 1947. """
docs = [Document(page_content=text)]

# Uses default map_reduce prompts under the hood
sum_chain = load_summarize_chain(llm, chain_type="map_reduce")
result = sum_chain.invoke({"input_documents": docs})
print(result["output_text"])


Pakistan is a South Asian country with a population of over 241.5 million, sharing borders with several countries, and boasting a rich history influenced by ancient cultures and empires.


In [11]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

# Pull a ready-made editing prompt from the Hub
prompt = hub.pull("bagatur/mattschumer-professional-editor")  # no custom prompt authored by us
print("Needs variables:", prompt.input_variables)              # e.g. ['input'] or ['text']

# Build a simple chain: Hub Prompt → LLM → string
edit_chain = prompt | llm | StrOutputParser()

sample_text = "This sentence have many mistake and not very clear what I means."
# pass in the right key from prompt.input_variables, usually 'input' or 'text'
print(edit_chain.invoke({"text": sample_text}))

Needs variables: ['text']
### Meaning
$meaning_bulleted_summary
- The sentence contains many errors.
- It is unclear what the sentence is trying to convey.

### Round 1
    ## Draft
        ``This sentence have many mistake and not very clear what I means.``
    ## Reflection
        The sentence is unclear and contains grammatical errors. It seems to be saying that the sentence itself has mistakes and is unclear, but the wording is confusing. To improve it, we need to rephrase it in a way that clearly conveys the intended meaning.

### Round 2
    ## Draft
        ``The sentence contains many errors and is unclear in its meaning.``
    ## Reflection
        The revised sentence is clearer, but it still doesn't fully convey the intended meaning. The phrase "in its meaning" is a bit awkward. We could rephrase it to make it more concise and natural-sounding.

### Round 3
    ## Draft
        ``The sentence is unclear and contains several errors.``
    ## Reflection
        This revised s

In [12]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser

clf_prompt = hub.pull("hwchase17/classify-article")
print("Needs variables:", clf_prompt.input_variables)  # usually ['text']

clf_chain = clf_prompt | llm | StrOutputParser()

article = "NVIDIA unveiled a new GPU architecture aimed at accelerating AI workloads..."
label = clf_chain.invoke({"text": article})
print("Predicted label:", label)


Needs variables: ['text']
Predicted label: The correct answer is: technology. 

This is because the text mentions a specific technology-related term, "GPU architecture," which is related to computer hardware and its applications in accelerating AI workloads.


In [13]:
from langchain import hub
tmpl = hub.pull("bagatur/mattschumer-professional-editor")
print(tmpl.input_variables)  # then pass a dict matching those keys


['text']


### 5- Decide at runtime: Chain or Agent?

Use a tiny router chain to decide if the query is simple (chain) or needs tools (agent).
(For class demos, this shows how to pick the right path.)

In [14]:
# Router prompt: return either "simple" or "needs_tools"
router_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "Decide routing based on query complexity:\n"
     "- Output 'simple' for: definitions, explanations, rewrites, translations\n"
     "- Output 'needs_tools' for: math calculations, current events, specific facts about places/people, data lookups\n"
     "Examples:\n"
     "- 'Explain machine learning' → simple\n"
     "- 'What is 15 * 23?' → needs_tools\n"
     "- 'Facts about Tokyo' → needs_tools\n"
     "Only respond with 'simple' or 'needs_tools'."),
    ("human", "{query}")
])
router_chain = router_prompt | llm | StrOutputParser()

def run_app(user_query: str):
    route = router_chain.invoke({"query": user_query}).strip().lower()
    print("route ", route)
    if "needs_tools" in route:
        print(f" Using AGENT for: '{user_query}'")
        # Directly invoke the agent executor
        return agent.invoke({"input": user_query})["output"]
    else:
        print(f" Using CHAIN for: '{user_query}'")
        return single_step_chain.invoke({"topic": user_query})

# print(run_app("Explain embeddings in a paragraph."))             # likely chain
print(run_app("What is 9*13 and a fact about Islamabad?"))      # likely agent

route  needs_tools
needs_tools
 Using AGENT for: 'What is 9*13 and a fact about Islamabad?'


> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 9, 'b': 13}`


117
Invoking: `wiki_search` with `{'query': 'Islamabad'}`


Page: Islamabad
Summary: Islamabad ( ; Urdu: اسلام‌آباد, romanized: Islāmābād, lit. 'City of Islam', [ɪsˈlɑːmɑːbɑːd] ) is the capital city of Pakistan. It is the country's tenth-most populous city with a population of over 1.1 million,  and is federally administered by the Pakistani government as part of the Islamabad Capital Territory. Built as a planned city in the 1960s and established in 1967, it replaced Karachi as Pakistan's national capital.
The Greek architect Constantinos Apostolou Doxiadis developed Islamabad's master plan, in which he divided it into eight zones; the city comprises administrative, diplomatic enclave, residential areas, educational and industrial sectors, commercial areas, as well as rural and green areas administered by t

## E- Indexes

Indexes in LangChain consists of the following components:
- Document Loader
- Text Splitter
- Vector Store
- Retrievers


## F- Memory

LangChain offers the following types of memory:
- ConversationBufferMemory
- ConversationBufferWindowMemory
- Summarizer-Based Memory
- Custom Memory